# Caso Práctico: Extracción Automática de Entidades en Tickets de Soporte para "TechSupportPro"

**Empresa Ficticia**: TechSupportPro Contexto: "TechSupportPro" ofrece software y hardware a miles de empresas. Su departamento de soporte técnico recibe cientos de tickets (correos electrónicos y formularios web) cada hora.

Actualmente, un equipo de "triaje" lee cada ticket manualmente para identificar la información clave y rellenar los campos en su sistema interno (como Zendesk o Jira). Este proceso es lento y propenso a errores. La información que necesitan extraer es:

- PRODUCTO: El nombre del software o hardware que falla (p.ej., "Firewall Serie XG", "App Agente v3.1").

- ERROR_CODE: Códigos de error específicos (p.ej., "0x00A45F", "E_CONN_REFUSED").

- OS: El sistema operativo del cliente (p.ej., "Windows Server 2022", "Ubuntu 22.04").

**El Desafío**: La empresa quiere construir un modelo de IA que lea el texto no estructurado de un ticket y etiquete automáticamente cada palabra (o "token") que corresponda a una de estas entidades. Esto se conoce como un problema de Reconocimiento de Entidades Nombradas (NER) o Clasificación de Tokens.

Por ejemplo:

Ticket: "Mi Firewall Serie XG [PRODUCTO] da el error 0x00A45F [ERROR_CODE] en Windows Server 2022 [OS]."

## Enunciado del Problema y Tarea

Tu objetivo es entrenar un modelo Transformer (como BERT-base-cased, DistilBert o cualquier otro que hayas utilizado en clase) para que funcione como un extractor de entidades (NER). El modelo debe tomar una frase y devolver una etiqueta para cada token de esa frase.

**Conjunto de Datos:** Se te proporciona (o generarás) un archivo tech_support_tickets_ner.csv. Este formato es diferente al problema anterior. En lugar de una etiqueta por reseña, tienes una etiqueta por palabra.

El CSV tiene 3 columnas:

- ticket_id: Identificador de la frase (un mismo ticket puede tener varias frases).

- tokens: Una lista de palabras (ya tokenizadas por espacio) que forman la frase.

- ner_tags: Una lista de etiquetas (en formato IOB2) que se corresponde exactamente con la lista de tokens.

**Esquema de Etiquetado (IOB2):**

- O: (Outside) La palabra no es ninguna entidad de interés.

- B-PROD: (Beginning) La palabra es el comienzo de una entidad de tipo Producto.

- I-PROD: (Inside) La palabra es la continuación de una entidad de tipo Producto.

- B-ERR: (Beginning) Comienzo de un Error Code.

- I-ERR: (Inside) Continuación de un Error Code.

- B-OS: (Beginning) Comienzo de un Sistema Operativo.

- I-OS: (Inside) Continuación de un Sistema Operativo.

Ejemplo de una fila del CSV: | ticket_id | tokens | ner_tags | | :--- | :--- | :--- | | tkt_001 | ['El', 'Firewall', 'Serie', 'XG', 'falla', 'con', 'E_CONN_REFUSED'] | ['O', 'B-PROD', 'I-PROD', 'I-PROD', 'O', 'O', 'B-ERR'] |

## Requisitos del Proyecto

1. Carga y Preparación de Datos:

- Cargar el .csv tech_support_tickets_ner.csv. Las columnas tokens y ner_tags se cargarán como strings. Deberás convertirlas de nuevo a listas.

- Crear los mapeos (diccionarios) entre las etiquetas (ej. 'B-PROD') y sus IDs numéricos (ej. 2).

2. Tokenización y Alineación (El Reto de NER):

- Elegir un tokenizador pre-entrenado (ej. BertTokenizerFast).

- Reto Clave: El tokenizador (WordPiece/BPE) dividirá palabras que en tus datos originales eran un solo token. Por ejemplo, "Firewall" puede seguir siendo "Firewall", pero "E_CONN_REFUSED" podría dividirse en ['E', '_', 'CONN', '_', 'REFUSED'].

- Debes implementar la lógica para alinear tus ner_tags con los nuevos sub-tokens generados por el tokenizador. La regla estándar es:

 La etiqueta B- solo se aplica al primer sub-token.

 Las etiquetas I- se aplican a los sub-tokens siguientes de la misma palabra.

 Los tokens especiales (como [CLS], [SEP]) deben recibir una etiqueta especial (p.ej., -100) para que sean ignorados por la función de pérdida.

3. Configuración del Modelo:

- Cargar un modelo Transformer pre-entrenado para Clasificación de Tokens (ej. BertForTokenClassification, AutoModelForTokenClassification o cualquier otro que hayas utilizado en clase).

- Asegúrate de configurarlo con el número correcto de etiquetas (IDs) que has definido.

4. Entrenamiento (Fine-Tuning):

- Configurar TrainingArguments y el Trainer.

- Entrenar el modelo (fine-tuning).

5. Evaluación:

- Utilizar el conjunto de prueba para evaluar el rendimiento.

- Importante: La métrica de "accuracy" simple no es útil aquí. Debes usar una biblioteca como seqeval (el estándar de oro para NER) para calcular la Precisión, Recall y F1-Score a nivel de entidad (no de token).

6. Función de Inferencia:

- Crear una función final extraer_entidades(texto) que reciba un string de texto crudo, lo procese (tokenice, prediga, y agrupe los sub-tokens) y devuelva una lista de las entidades encontradas.

Ejemplo de salida: [{'entidad': 'Firewall Serie XG', 'tipo': 'PRODUCTO'}, {'entidad': 'E_CONN_REFUSED', 'tipo': 'ERROR_CODE'}]